In [1]:
import rasterio
import rioxarray as riox
import rasterio.plot
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling
from shapely.geometry import box
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import numpy as np
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import log

ModuleNotFoundError: No module named 'rioxarray'

In [2]:
logger = log.get_logger(__name__)

In [3]:
def load_raster(path):
    # Load raster data
    img = rasterio.open(path)
    logger.info(f"Loaded image: {path}")
    logger.info(f"Image channels: {img.count}")
    logger.info(f"Image size: {img.width}x{img.height}")
    logger.info(f"Image crs: {img.crs}")
    logger.info(f"Image bounds: {img.bounds}")
    logger.info(f"Image transform: {img.transform}")
    return img


def sync_crs(gdf, rasterimg) -> bool: 
    if gdf.crs != rasterimg.crs:
        gdf = gdf.set_crs(str(rasterimg.crs))
    return gdf

def load_data(raw_dir, fn_ms, fn_rgb, fn_dsm, fn_dtm, fn_shp):
    imgs = {}
    # 10 bands of Advanced Camera image (multispectral)
    # 3 bands of Normal Camera image
    # 1 band of Digital Surface Model
    # 1 band of Digital Terrain Model
    imgs["ms"] = load_raster(f"{raw_dir}{fn_ms}")
    imgs["rgb"] = load_raster(f"{raw_dir}{fn_rgb}")
    imgs["dsm"] = load_raster(f"{raw_dir}{fn_dsm}")
    imgs["dtm"] = load_raster(f"{raw_dir}{fn_dtm}")
    # Load shapefile
    gdf = gpd.read_file(f"{raw_dir}{fn_shp}")
    return imgs, gdf


def get_bbox(imgs: dict):
    box1 = box(*imgs["ms"].bounds)
    box2 = box(*imgs["rgb"].bounds)
    box3 = box(*imgs["dsm"].bounds)
    box4 = box(*imgs["dtm"].bounds)

    intersect = box1.intersection(box2)
    intersect = intersect.intersection(box3)
    intersect = intersect.intersection(box4)

    return intersect


def clip_raster_to_bounds(name, path_int, raster, target_bounds):
    # Create a polygon from the target bounds
    target_polygon = box(*target_bounds)

    # Clip the raster using the target polygon
    clipped_data, clipped_transform = mask(raster, [target_polygon], crop=True)

    # Update the raster metadata
    clipped_meta = raster.meta.copy()
    clipped_meta.update({
        'transform': clipped_transform,
        'height': clipped_data.shape[1],
        'width': clipped_data.shape[2]
    })

    # Create a new raster dataset with the clipped data and metadata
    with rasterio.open(f"{path_int}clipped/{name}.tif", "w", **clipped_meta) as dst:
        dst.write(clipped_data)

    # Return the clipped raster dataset
    return rasterio.open(f"{path_int}clipped/{name}.tif")


def resample_raster(name, path_int, raster, target_resolution):
    scale_x = target_resolution / raster.res[0]
    scale_y = target_resolution / raster.res[1]

    new_width = int(raster.width * scale_x)
    new_height = int(raster.height * scale_y)

    profile = raster.profile.copy()
    profile.update(width=new_width, height=new_height)

    # Update the transform property to maintain the original extent
    new_transform = raster.transform * rasterio.Affine.scale(scale_x, scale_y)
    profile.update(transform=new_transform)

    with rasterio.open(f"{path_int}resampled/{name}.tif", "w", **profile) as dst:
        for i in range(1, raster.count + 1):
            data = raster.read(i, out_shape=(new_height, new_width), resampling=Resampling.bilinear)
            dst.write(data, i)

    return rasterio.open(f"{path_int}resampled/{name}.tif")


def align_rasters(name, path_int, src_raster, ref_raster, ):
    profile = ref_raster.profile.copy()
    aligned_data = []

    for i in range(1, src_raster.count + 1):
        src_data = src_raster.read(i)
        ref_data = np.empty((ref_raster.height, ref_raster.width), dtype=src_data.dtype)
        reproject(
            src_data,
            ref_data,
            src_transform=src_raster.transform,
            src_crs=src_raster.crs,
            dst_transform=ref_raster.transform,
            dst_crs=ref_raster.crs,
            resampling=Resampling.nearest
        )
        aligned_data.append(ref_data)

    profile.update(count=len(aligned_data))

    with rasterio.open(f"{path_int}aligned/{name}.tif", "w", **profile) as dst:
        for i, data in enumerate(aligned_data, start=1):
            dst.write(data, i)

    return rasterio.open(f"{path_int}aligned/{name}.tif")

In [4]:
path_raw = "../data/01_raw/data_masked_letaba/"
fn_ms = "ortho_multispect_15cm_aoi.tif"
fn_rgb = "ortho_rgb_7cm_aoi.tif"
fn_dsm = "dsm_28cm_mask.tif"
fn_dtm = "dtm_28cm_mask.tif"
fn_shp = "letaba_invasives_june21_utm.shp"
path_int = "../data/02_intermediate/"

In [5]:
imgs, gdf = load_data(path_raw, fn_ms, fn_rgb, fn_dsm, fn_dtm, fn_shp)


02-Nov-23 00:48:53 - INFO - Loaded image: ../data/01_raw/data_masked_letaba/ortho_multispect_15cm_aoi.tif
02-Nov-23 00:48:53 - INFO - Image channels: 10
02-Nov-23 00:48:53 - INFO - Image size: 10101x10592
02-Nov-23 00:48:53 - INFO - Image crs: EPSG:32736
02-Nov-23 00:48:53 - INFO - Image bounds: BoundingBox(left=362292.84996986337, bottom=7356585.965293335, right=363843.98982419644, top=7358212.504589749)
02-Nov-23 00:48:53 - INFO - Image transform: | 0.15, 0.00, 362292.85|
| 0.00,-0.15, 7358212.50|
| 0.00, 0.00, 1.00|
02-Nov-23 00:48:53 - INFO - Loaded image: ../data/01_raw/data_masked_letaba/ortho_rgb_7cm_aoi.tif
02-Nov-23 00:48:53 - INFO - Image channels: 4
02-Nov-23 00:48:53 - INFO - Image size: 22379x23468
02-Nov-23 00:48:53 - INFO - Image crs: EPSG:32736
02-Nov-23 00:48:53 - INFO - Image bounds: BoundingBox(left=362292.7879142576, bottom=7356585.87461337, right=363843.97933615686, top=7358212.5496497555)
02-Nov-23 00:48:53 - INFO - Image transform: | 0.07, 0.00, 362292.79|
| 0.00

In [6]:
intersecting_box = get_bbox(imgs)
target_bounds = intersecting_box.bounds

In [7]:
rgb_clipped = clip_raster_to_bounds('rgb', path_int, imgs['rgb'], target_bounds)
ms_clipped = clip_raster_to_bounds('ms', path_int, imgs['ms'], target_bounds)
dsm_clipped = clip_raster_to_bounds('dsm', path_int, imgs['dsm'], target_bounds)
dtm_clipped = clip_raster_to_bounds('dtm', path_int, imgs['dtm'], target_bounds)

In [8]:
target_resolution = 0.28

In [ ]:
#downsample raster
rgb_down_sampled = rgb_clipped.rio.reproject(raster.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.bilinear)
 

In [9]:
rgb_resampled = resample_raster('rgb', path_int, rgb_clipped, target_resolution)
ms_resampled = resample_raster('ms', path_int, ms_clipped, target_resolution)

In [10]:
rgb_aligned = align_rasters('rgb', path_int, rgb_resampled, dsm_clipped)
ms_aligned = align_rasters('ms', path_int, ms_resampled, dsm_clipped)

In [11]:
logger.info(f"{rgb_aligned.transform}")
logger.info(f"{ms_aligned.transform}")
logger.info(f"{dsm_clipped.transform}")
logger.info(f"{dtm_clipped.transform}")

02-Nov-23 01:01:00 - INFO - | 0.28, 0.00, 362292.79|
| 0.00,-0.28, 7358212.35|
| 0.00, 0.00, 1.00|
02-Nov-23 01:01:00 - INFO - | 0.28, 0.00, 362292.79|
| 0.00,-0.28, 7358212.35|
| 0.00, 0.00, 1.00|
02-Nov-23 01:01:00 - INFO - | 0.28, 0.00, 362292.79|
| 0.00,-0.28, 7358212.35|
| 0.00, 0.00, 1.00|
02-Nov-23 01:01:00 - INFO - | 0.28, 0.00, 362292.79|
| 0.00,-0.28, 7358212.35|
| 0.00, 0.00, 1.00|


In [12]:
 rgb_aligned.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 5594, 'height': 5866, 'count': 4, 'crs': CRS.from_epsg(32736), 'transform': Affine(0.2772579991658268, 0.0, 362292.7863346958,
       0.0, -0.27725799938891194, 7358212.348826846), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}

In [13]:
dtm_clipped.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 5594, 'height': 5866, 'count': 1, 'crs': CRS.from_epsg(32736), 'transform': Affine(0.2772579991658268, 0.0, 362292.7863346958,
       0.0, -0.27725799938891194, 7358212.348826846), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}

In [ ]:
def clip_gdf(gdf, bounds):
    clipped_gdf = gdf[
        (gdf.geometry.x > bounds.left) & 
        (gdf.geometry.x < bounds.right) &
        (gdf.geometry.y > bounds.bottom) &
        (gdf.geometry.y < bounds.top)
    ]
    return clipped_gdf

In [ ]:
gdf = gpd.read_file(f"{path_raw}{fn_shp}")

In [ ]:
clipped_gdf = clip_gdf(gdf, rgb_aligned.bounds)

In [ ]:
gdf.shape

In [ ]:
clipped_gdf.shape

In [ ]:
clipped_gdf.Species.value_counts()

In [ ]:
def plot_raster(gdf, rasterimg):
    fig, ax = plt.subplots(figsize = (20,20))
    rasterio.plot.show(rasterimg, ax=ax)
    gdf.plot(column='Species',
                   categorical=True,
                   legend=True,
                   # markersize=45,
                   cmap="Set2",
                   ax=ax,
            aspect=1)
    ax.set_title("Letaba Points Subset")
    for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf.photoID):
        ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
    plt.show()

In [ ]:
# plot_raster(gdf, rgb_aligned)

In [ ]:
def plot_raster(path, name):
    with rasterio.open(f"{path}{name}.tif") as src:
        raster_arr = src.read(1)
    ep.plot_bands(raster_arr, title=name, cmap="Greys")
    plt.show()

In [ ]:
# # Open the file using a context manager ("with rio.open" statement)
# 

# ep.plot_bands(raster_arr, title="Your Title", cmap="Greys")
# plt.show()

In [ ]:
def plot_raster(path, name):
    with rasterio.open(f"{path}{name}.tif") as src:
        raster_arr = src.read(1)
        extent = rasterio.transform.array_bounds(src.height, src.width, src.transform)
    plt.imshow(raster_arr, cmap="Greys", extent=extent)
    plt.title(name)
    plt.show()

In [ ]:
plot_raster(f"{path_int}clipped/", name="rgb")

In [ ]:
plot_raster(f"{path_int}aligned/", name="rgb")